In [4]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob

In [2]:
run = 'parallel_new_input'
os.chdir('/home/smmrrr/TEM/TEM_Runs/'+run)

print(os.getcwd()) 


/home/smmrrr/TEM/TEM_Runs/parallel_new_input


In [20]:
output_paths = glob.glob(r'*.csv[0123456789]*')
input_paths = glob.glob('*.HVD*')

print(len(output_paths))
print(len(input_paths))

500
250


In [21]:
print(len(output_paths))
print(len(output_group))


500
500


In [48]:
cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
            , names = output_col_names)# output_paths

In [49]:
cohort_output.loc[0]

lon                                   -108.5
lat                                     48.5
variable                                NEP 
cohort_number                              1
stand_age                               3000
potential_veg                             13
current_veg                               13
subtype                                   13
community_type                             8
silt_clay                              56.98
lc_state                                   0
land_area                               2038
cohort_area                             2038
year                                    1750
annual_sum                               0.0
monthly_maximum                         19.4
monthly_mean                             0.0
monthly_minimum                        -16.5
Jan                                    -4.74
Feb                                    -1.64
Mar                                    -3.08
Apr                                     0.56
May       

In [47]:

output_col_names = ['lon'
,'lat'
,'variable'
,'cohort_number'
,'stand_age'
,'potential_veg'
,'current_veg'
,'subtype'
,'community_type'
,'silt_clay'  ###check this order
,'lc_state'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

input_col_names = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

In [23]:

output_group = [re.findall('[0-9]+$', w) for w in output_paths]
output_group = [item for sublist in output_group for item in sublist]

output_var = [re.findall('AVLN|NCE|NPP|SOLC|VEGC|GPP|NEP|SMOIST|SOLN|VEGN', w) for w in output_paths]
output_var = [item for sublist in output_var for item in sublist]

input_group = [re.findall('[0-9]+$', w) for w in input_paths]
input_group = [item.upper() for sublist in input_group for item in sublist]

input_var = [re.findall('PREC|CLDS|PAR|TAIR|VPR', w) for w in input_paths]
input_var = [item for sublist in input_var for item in sublist]


In [24]:
# input_var

In [15]:
# output_group

In [25]:
#make tables with the relevant read in info

input_data = pd.DataFrame(
{'input_paths': input_paths,
'input_group': input_group,
 'input_var': input_var
}
)

output_data = pd.DataFrame(
{'output_paths': output_paths,
'output_group': output_group,
 'output_var': output_var
}
)

In [26]:
input_data

,input_paths,input_group,input_var
0,CLDS.HVD544,544,CLDS
1,PAR.HVD2244,2244,PAR
2,TAIR.HVD2720,2720,TAIR
3,PAR.HVD2040,2040,PAR
4,PREC.HVD952,952,PREC
...,...,...,...
245,VPR.HVD1360,1360,VPR
246,CLDS.HVD1700,1700,CLDS
247,TAIR.HVD1292,1292,TAIR
248,TAIR.HVD3196,3196,TAIR


In [57]:
i

1

In [27]:

input_data = pd.pivot(input_data, index = 'input_group', columns = 'input_var', values = 'input_paths').reset_index()

read_in_data = input_data.merge(output_data, left_on = 'input_group', right_on = 'output_group')

In [ ]:
read_in_data.to_csv('/home/smmrrr/TEM_Analysis/test_parallel_run/all_parallel_variable_info.csv')

In [29]:
read_in_data = read_in_data.loc[read_in_data['output_var'] == 'NEP'].reset_index()

In [30]:
read_in_data

,index,input_group,CLDS,PAR,PREC,TAIR,VPR,output_paths,output_group,output_var
0,7,0,CLDS.HVD0,PAR.HVD0,PREC.HVD0,TAIR.HVD0,VPR.HVD0,NEP_LULCC.csv0,0,NEP
1,10,1020,CLDS.HVD1020,PAR.HVD1020,PREC.HVD1020,TAIR.HVD1020,VPR.HVD1020,NEP_LULCC.csv1020,1020,NEP
2,24,1088,CLDS.HVD1088,PAR.HVD1088,PREC.HVD1088,TAIR.HVD1088,VPR.HVD1088,NEP_LULCC.csv1088,1088,NEP
3,34,1156,CLDS.HVD1156,PAR.HVD1156,PREC.HVD1156,TAIR.HVD1156,VPR.HVD1156,NEP_LULCC.csv1156,1156,NEP
4,40,1224,CLDS.HVD1224,PAR.HVD1224,PREC.HVD1224,TAIR.HVD1224,VPR.HVD1224,NEP_LULCC.csv1224,1224,NEP
5,55,1292,CLDS.HVD1292,PAR.HVD1292,PREC.HVD1292,TAIR.HVD1292,VPR.HVD1292,NEP_LULCC.csv1292,1292,NEP
6,66,136,CLDS.HVD136,PAR.HVD136,PREC.HVD136,TAIR.HVD136,VPR.HVD136,NEP_LULCC.csv136,136,NEP
7,74,1360,CLDS.HVD1360,PAR.HVD1360,PREC.HVD1360,TAIR.HVD1360,VPR.HVD1360,NEP_LULCC.csv1360,1360,NEP
8,80,1428,CLDS.HVD1428,PAR.HVD1428,PREC.HVD1428,TAIR.HVD1428,VPR.HVD1428,NEP_LULCC.csv1428,1428,NEP
9,94,1496,CLDS.HVD1496,PAR.HVD1496,PREC.HVD1496,TAIR.HVD1496,VPR.HVD1496,NEP_LULCC.csv1496,1496,NEP


In [32]:
# ## TEM PFT info
# # forest_vegs = [9, 4, 16, 33, 19, 10 ]
# # forest_veg_names = ['needle-leaf evergreen temperate','boreal','broadleaved evergreen tree tropical'
# #                    ,'broadleaved evergreen tree temperate','broadleaved deciduous tree tropical'
# #                    ,'broadleaved deciduous tree temperate']


# ouput_data_path = '/home/smmrrr/TEM_Analysis/usa_runs_processed/'

# # for i in range(len(read_in_data)):
# for i in range(len(read_in_data)):

# ###read in cohort output
#     cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
#                 , names = output_col_names)

#     ##read in precipitation files
#     prec = pd.read_csv(read_in_data.loc[i, 'PREC']
#                , names = input_col_names)
#     par = pd.read_csv(read_in_data.loc[i, 'PAR']
#                , names = input_col_names)
#     clds = pd.read_csv(read_in_data.loc[i, 'CLDS']
#                , names = input_col_names)
#     tair = pd.read_csv(read_in_data.loc[i, 'TAIR']
#                , names = input_col_names)
#     vpr = pd.read_csv(read_in_data.loc[i, 'VPR']
#                , names = input_col_names)


#     ## shape input datasets wide to long
#     prec = pd.melt(prec, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
#     'Jan',
#     'Feb',
#     'Mar',
#     'Apr',
#     'May',
#     'Jun',
#     'Jul',
#     'Aug',
#     'Sep',
#     'Oct',
#     'Nov',
#     'Dec'], 
#              var_name='month', value_name='prec'
#             ,col_level=0)

#     par = pd.melt(par, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
#     'Jan',
#     'Feb',
#     'Mar',
#     'Apr',
#     'May',
#     'Jun',
#     'Jul',
#     'Aug',
#     'Sep',
#     'Oct',
#     'Nov',
#     'Dec'], 
#              var_name='month', value_name='par'
#             ,col_level=0)

#     clds = pd.melt(clds, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
#     'Jan',
#     'Feb',
#     'Mar',
#     'Apr',
#     'May',
#     'Jun',
#     'Jul',
#     'Aug',
#     'Sep',
#     'Oct',
#     'Nov',
#     'Dec'], 
#              var_name='month', value_name='clds'
#             ,col_level=0)

#     tair = pd.melt(tair, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
#     'Jan',
#     'Feb',
#     'Mar',
#     'Apr',
#     'May',
#     'Jun',
#     'Jul',
#     'Aug',
#     'Sep',
#     'Oct',
#     'Nov',
#     'Dec'], 
#              var_name='month', value_name='tair'
#             ,col_level=0)
    
#     vpr = pd.melt(vpr, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
#     'Jan',
#     'Feb',
#     'Mar',
#     'Apr',
#     'May',
#     'Jun',
#     'Jul',
#     'Aug',
#     'Sep',
#     'Oct',
#     'Nov',
#     'Dec'], 
#              var_name='month', value_name='vpr'
#             ,col_level=0)

#     ##merge climate variables together
#     climate_vars = prec.merge(
#                 par,on = ['lon','lat', 'Area', 'year', 'month']
#                 ).merge(
#                 tair ,on = ['lon','lat', 'Area', 'year', 'month']                           
#                 ).merge(
#                 clds ,on = ['lon','lat', 'Area', 'year', 'month']                           
#                 ).merge(
#                 vpr ,on = ['lon','lat', 'Area', 'year', 'month']                           
#                 )

#     ##subset to forest veg types 
#     # cohort_output = cohort_output.loc[cohort_output['current_veg'].isin(forest_vegs)] 
# forest_vegs = [9, 4, 16, 33, 19, 10 ]
#     ##melt cohort output
#     cohort_output = pd.melt(cohort_output, id_vars = ['lon','lat', 'cohort_area', 'land_area', 'year', 'current_veg', 'stand_age'], value_vars = [
#     'Jan',
#     'Feb',
#     'Mar',
#     'Apr',
#     'May',
#     'Jun',
#     'Jul',
#     'Aug',
#     'Sep',
#     'Oct',
#     'Nov',
#     'Dec'], 
#              var_name='month', value_name=read_in_data.loc[i, 'output_var']
#             ,col_level=0)


#     agg_cohort_output = cohort_output.merge(
#                 climate_vars, how = 'left'
#         ,on = ['lon','lat', 'year', 'month']
#                 )


    
#     # agg_cohort_output.to_csv(ouput_data_path+'merged_dataset_'+read_in_data.loc[i, 'output_group']+'.csv', index=False)
#     agg_cohort_output.count()

#     cohort_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "cohort_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
#     land_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "land_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe

#     if (i==0):


#         merged_dataset_annual_loop = agg_cohort_output.groupby([
#                 'month','year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
#                                                                                       NCE_weighted=('NCE', cohort_area), 
#                                                                                    prec = ('prec', land_area),
#                                                                                    par = ('par', land_area),
#                                                                                    tair = ('tair', land_area),
#                                                                                    clds = ('clds', land_area),
#                                                                                    vpr = ('vpr', land_area))
                                                                                    
#     else:
                                                        
#         merged_dataset_annual_loop = pd.concat([merged_dataset_annual_loop,
#                               agg_cohort_output.groupby([
#                 'month','year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
#                                                                                       NCE_weighted=('NCE', cohort_area), 
#                                                                                    prec = ('prec', land_area),
#                                                                                    par = ('par', land_area),
#                                                                                    tair = ('tair', land_area),
#                                                                                    clds = ('clds', land_area),
#                                                                                    vpr = ('vpr', land_area))])
   
#     merged_dataset_annual_loop.to_csv(ouput_data_path+'merged_dataset_annual_loop.csv')
#     print(i)
#     print(merged_dataset_annual_loop.count())


In [35]:
read_in_data.loc[i, 'output_var'] + '_weighted'

'NEP_weighted'

In [64]:
i = 2

read_in_data = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/test_parallel_run/all_parallel_variable_info.csv')
print(read_in_data.loc[i])


input_group                  0
CLDS                 CLDS.HVD0
PAR                   PAR.HVD0
PREC                 PREC.HVD0
TAIR                 TAIR.HVD0
VPR                   VPR.HVD0
output_paths    NCE_LULCC.csv0
output_group                 0
output_var                 NCE
Name: 2, dtype: object


In [66]:

output_col_names = ['lon'
,'lat'
,'variable'
,'cohort_number'
,'stand_age'
,'potential_veg'
,'current_veg'
,'subtype'
,'community_type'
,'silt_clay'  ###check this order
,'lc_state'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

input_col_names = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

###read in cohort output
cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
            , names = output_col_names)

##read in precipitation files
prec = pd.read_csv(read_in_data.loc[i, 'PREC']
           , names = input_col_names)
par = pd.read_csv(read_in_data.loc[i, 'PAR']
           , names = input_col_names)
clds = pd.read_csv(read_in_data.loc[i, 'CLDS']
           , names = input_col_names)
tair = pd.read_csv(read_in_data.loc[i, 'TAIR']
           , names = input_col_names)
vpr = pd.read_csv(read_in_data.loc[i, 'VPR']
           , names = input_col_names)

print('data read in')
## shape input datasets wide to long
prec = pd.melt(prec, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='prec'
        ,col_level=0)

par = pd.melt(par, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='par'
        ,col_level=0)

clds = pd.melt(clds, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='clds'
        ,col_level=0)

tair = pd.melt(tair, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='tair'
        ,col_level=0)

vpr = pd.melt(vpr, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='vpr'
        ,col_level=0)

##merge climate variables together
climate_vars = prec.merge(
            par,on = ['lon','lat', 'Area', 'year', 'month']
            ).merge(
            tair ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            clds ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            vpr ,on = ['lon','lat', 'Area', 'year', 'month']                           
            )

##subset to forest veg types 
# cohort_output = cohort_output.loc[cohort_output['current_veg'].isin(forest_vegs)] 
print('clim processed')
##melt cohort output
cohort_output = pd.melt(cohort_output, id_vars = ['lon','lat', 'cohort_area', 'land_area', 'year', 'current_veg', 'stand_age'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='value'
        ,col_level=0)
cohort_output = cohort_output[cohort_output['cohort_area']>0]
print('cohort processed')

agg_cohort_output = cohort_output.merge(
            climate_vars, how = 'left'
    ,on = ['lon','lat', 'year', 'month']
            )

print('climate and cohort merged')
print(agg_cohort_output.count())

cohort_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "cohort_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
land_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "land_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe



merged_dataset = agg_cohort_output.groupby([
        'month','year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                              value_weighted=('value', cohort_area), 
                                                                           prec = ('prec', land_area),
                                                                           par = ('par', land_area),
                                                                           tair = ('tair', land_area),
                                                                           clds = ('clds', land_area),
                                                                           vpr = ('vpr', land_area))
merged_dataset.rename(columns={'value_weighted':read_in_data.loc[i, 'output_var'] + '_weighted'}, inplace = True)
print('rename')


# agg_cohort_output.to_csv(ouput_data_path+'merged_dataset_'+read_in_data.loc[i, 'output_group']+'.csv', index=False)
print(merged_dataset.count())




merged_dataset.to_csv(ouput_data_path+read_in_data.loc[i, 'output_var']+'_'+read_in_data.loc[i, 'output_group']+'_'+'processed.csv')


data read in
clim processed
cohort processed
climate and cohort merged
lon            24547008
lat            24547008
cohort_area    24547008
land_area      24547008
year           24547008
current_veg    24547008
stand_age      24547008
month          24547008
value          24547008
Area           24547008
prec           24547008
par            24547008
tair           24547008
clds           24547008
vpr            24547008
dtype: int64
rename
cohort_area     1332576
total_area      1332576
NCE_weighted    1332576
prec            1332576
par             1332576
tair            1332576
clds            1332576
vpr             1332576
dtype: int64


TypeError: can only concatenate str (not "numpy.int64") to str

In [70]:
merged_dataset.reset_index().loc[0]

month                  Apr
year                  1750
current_veg              8
stand_age              425
cohort_area             32
total_area           30456
NCE_weighted     19.866563
prec             92.410796
par             122.751681
tair              8.392067
clds             32.523273
vpr               7.910908
Name: 0, dtype: object

In [75]:
merged_dataset= merged_dataset.reset_index()

In [76]:
forest_vegs = [4, 5, 6, 8, 9, 10, 11, 16, 17, 18, 19, 20, 25, 33]
merged_dataset.loc[merged_dataset['current_veg'].isin(forest_vegs)]

,month,year,current_veg,stand_age,cohort_area,total_area,NCE_weighted,prec,par,tair,clds,vpr
0,Apr,1750,8,425,32,30456,19.866563,92.410796,122.751681,8.392067,32.523273,7.910908
1,Apr,1750,8,3000,21544,43702,10.452611,125.611144,117.464976,7.698563,38.880065,7.826283
2,Apr,1750,9,425,69,27868,29.883043,107.703179,117.452551,5.576679,40.882790,6.824566
3,Apr,1750,9,3000,96495,96942,19.157999,225.426687,110.089753,5.252081,47.635679,7.083233
4,Apr,1750,10,3000,4487,4539,-19.255837,97.577330,111.223640,5.642256,47.321062,6.695924
...,...,...,...,...,...,...,...,...,...,...,...,...
1332042,Sep,2014,33,100,18,2359,-10.580000,58.100000,114.000000,20.900000,13.500000,11.600000
1332043,Sep,2014,33,101,18,4718,-10.580000,58.100000,114.000000,20.900000,13.500000,11.600000
1332044,Sep,2014,33,102,17,2359,-10.570000,58.100000,114.000000,20.900000,13.500000,11.600000
1332045,Sep,2014,33,103,3,2359,-10.570000,58.100000,114.000000,20.900000,13.500000,11.600000


In [77]:
#     # cohort_output = cohort_output.loc[cohort_output['current_veg'].isin(forest_vegs)] 
merged_dataset= merged_dataset.reset_index()
merged_dataset.to_csv(ouput_data_path+read_in_data.loc[i, 'output_var']+'_'+str(read_in_data.loc[i, 'output_group'])+'_'+'processed.csv')
forest_vegs = [4, 5, 6, 8, 9, 10, 11, 16, 17, 18, 19, 20, 25, 33]
forests = merged_dataset.loc[merged_dataset['current_veg'].isin(forest_vegs)]
print(forests.count())
forests.to_csv(ouput_data_path+'forests_'+read_in_data.loc[i, 'output_var']+'_'+str(read_in_data.loc[i, 'output_group'])+'_'+'processed.csv')


index           487776
month           487776
year            487776
current_veg     487776
stand_age       487776
cohort_area     487776
total_area      487776
NCE_weighted    487776
prec            487776
par             487776
tair            487776
clds            487776
vpr             487776
dtype: int64


In [65]:
cohort_output = cohort_output[cohort_output['cohort_area']>0]

,lon,lat,cohort_area,land_area,year,current_veg,stand_age,month,value
20366,-124.5,40.0,0,1846,1840,51,0,Jan,0.01
20389,-124.5,40.0,0,1846,1841,51,1,Jan,0.02
20412,-124.5,40.0,0,1846,1842,51,2,Jan,0.08
20435,-124.5,40.0,0,1846,1843,51,3,Jan,0.13
20459,-124.5,40.0,0,1846,1844,51,4,Jan,0.18
...,...,...,...,...,...,...,...,...,...
24271345,-123.0,38.5,0,2410,2010,8,3000,Dec,50.20
24271860,-123.0,38.5,0,2410,2011,8,3000,Dec,51.17
24272377,-123.0,38.5,0,2410,2012,8,3000,Dec,52.14
24272898,-123.0,38.5,0,2410,2013,8,3000,Dec,51.42


In [58]:
cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
            , names = output_col_names)

##read in precipitation files
prec = pd.read_csv(read_in_data.loc[i, 'PREC']
           , names = input_col_names)
par = pd.read_csv(read_in_data.loc[i, 'PAR']
           , names = input_col_names)
clds = pd.read_csv(read_in_data.loc[i, 'CLDS']
           , names = input_col_names)
tair = pd.read_csv(read_in_data.loc[i, 'TAIR']
           , names = input_col_names)
vpr = pd.read_csv(read_in_data.loc[i, 'VPR']
           , names = input_col_names)

print('data read in')
## shape input datasets wide to long
prec = pd.melt(prec, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='prec'
        ,col_level=0)

par = pd.melt(par, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='par'
        ,col_level=0)

clds = pd.melt(clds, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='clds'
        ,col_level=0)

tair = pd.melt(tair, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='tair'
        ,col_level=0)

vpr = pd.melt(vpr, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='vpr'
        ,col_level=0)

##merge climate variables together
climate_vars = prec.merge(
            par,on = ['lon','lat', 'Area', 'year', 'month']
            ).merge(
            tair ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            clds ,on = ['lon','lat', 'Area', 'year', 'month']                           
            ).merge(
            vpr ,on = ['lon','lat', 'Area', 'year', 'month']                           
            )

##subset to forest veg types 
# cohort_output = cohort_output.loc[cohort_output['current_veg'].isin(forest_vegs)] 
print('clim processed')
##melt cohort output
cohort_output = pd.melt(cohort_output, id_vars = ['lon','lat', 'cohort_area', 'land_area', 'year','variable', 'current_veg', 'stand_age'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='value'
        ,col_level=0)

print('cohort processed')

agg_cohort_output = cohort_output.merge(
            climate_vars, how = 'left'
    ,on = ['lon','lat', 'year', 'month']
            )

print('climate and cohort merged')
print(agg_cohort_output.count())

cohort_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "cohort_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
land_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "land_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe



merged_dataset = agg_cohort_output.groupby([
        'month','year','current_veg', 'stand_age', 'variable']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                              value_weighted=('value', cohort_area), 
                                                                           prec = ('prec', land_area),
                                                                           par = ('par', land_area),
                                                                           tair = ('tair', land_area),
                                                                           clds = ('clds', land_area),
                                                                           vpr = ('vpr', land_area))
merged_dataset.rename(columns={'value_weighted':read_in_data.loc[i, 'output_var'] + '_weighted'}, inplace = True)
print('rename')


# agg_cohort_output.to_csv(ouput_data_path+'merged_dataset_'+read_in_data.loc[i, 'output_group']+'.csv', index=False)
print(merged_dataset.count())




merged_dataset.to_csv(ouput_data_path+read_in_data.loc[i, 'output_var']+'_'+read_in_data.loc[i, 'output_group']+'_'+'processed.csv')



data read in
clim processed
cohort processed
climate and cohort merged
lon            30499572
lat            30499572
cohort_area    30499572
land_area      30499572
year           30499572
variable       30499572
current_veg    30499572
stand_age      30499572
month          30499572
value          30499572
Area           30499572
prec           30499572
par            30499572
tair           30499572
clds           30499572
vpr            30499572
dtype: int64
rename
cohort_area     1006644
total_area      1006644
NEP_weighted    1006644
prec            1006644
par             1006644
tair            1006644
clds            1006644
vpr             1006644
dtype: int64


In [52]:
cohort_output = pd.melt(cohort_output, id_vars = ['lon','lat' ,'variable','cohort_area', 'land_area', 'year', 'current_veg', 'stand_age'], value_vars = [
'Jan',
'Feb',
'Mar',
'Apr',
'May',
'Jun',
'Jul',
'Aug',
'Sep',
'Oct',
'Nov',
'Dec'], 
         var_name='month', value_name='value'
        ,col_level=0)


agg_cohort_output = cohort_output.merge(
            climate_vars, how = 'left'
    ,on = ['lon','lat', 'year', 'month']
            )


In [53]:

# agg_cohort_output.to_csv(ouput_data_path+'merged_dataset_'+read_in_data.loc[i, 'output_group']+'.csv', index=False)
agg_cohort_output.count()

cohort_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "cohort_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
land_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "land_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe

# if (i==0):
merged_dataset = agg_cohort_output.groupby([
        'month','year','current_veg', 'stand_age', 'variable']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                              value_weighted=('value', cohort_area), 
                                                                           prec = ('prec', land_area),
                                                                           par = ('par', land_area),
                                                                           tair = ('tair', land_area),
                                                                           clds = ('clds', land_area),
                                                                           vpr = ('vpr', land_area))
merged_dataset.rename(columns={'value_weighted':read_in_data.loc[i, 'output_var'] + '_weighted'}, inplace = True)
print('rename')


rename


In [55]:
merged_dataset.reset_index().loc[0]

month                Apr
year                1750
current_veg            2
stand_age           3000
cohort_area         4863
total_area          4888
NEP_weighted   -0.150056
prec               21.75
par               131.35
tair               -2.95
clds                21.4
vpr                 0.55
Name: 0, dtype: object

In [56]:
len(merged_dataset)

1006644

In [40]:
climate_vars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216240 entries, 0 to 216239
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   lon     216240 non-null  float64
 1   lat     216240 non-null  float64
 2   Area    216240 non-null  int64  
 3   year    216240 non-null  int64  
 4   month   216240 non-null  object 
 5   prec    216240 non-null  float64
 6   par     216240 non-null  float64
 7   tair    216240 non-null  float64
 8   clds    216240 non-null  float64
 9   vpr     216240 non-null  float64
dtypes: float64(7), int64(2), object(1)
memory usage: 18.1+ MB
